# Import and Load Data

In [1]:
import numpy as np
import pandas as pd
from pandas import Series,DataFrame
from astropy.table import Table
from astropy.io import fits
import matplotlib.pyplot as plt
import h5py
from tqdm import tqdm
from IPython.display import display, Math

## modules I am still trying to learn how to use
import numexpr
import bottleneck

##
pd.set_option('display.float_format', lambda x: '%.3e' % x)
np.set_printoptions(formatter={'all':lambda x: '%.3e'% x})

## Functions to load data

In [9]:
mcal2=h5py.File('/lsst/troxel/balrog/balrog_mcal_stack-y3v02-0-riz-mcal-1.3.h5','r')['catalog']['unsheared'] #with neighbor
mcal3=h5py.File('/lsst/troxel/balrog/balrog_mcal_stack-y3v02-0-riz-noNB-mcal-1.3.h5','r')['catalog']['unsheared'] #no neighbor
#only matched balrog object griz
mcal4=h5py.File('/lsst/troxel/balrog/balrog_mcal_stack-y3v02-0-griz-mcal-1.3.h5','r')['catalog']['unsheared'] #with neighbor
mcal5=h5py.File('/lsst/troxel/balrog/balrog_mcal_stack-y3v02-0-griz-noNB-mcal-1.3.h5','r')['catalog']['unsheared'] #no neighbor


In [10]:
np.sum(np.unpackbits(np.uint8(mcal2['flags']), axis=0),axis=0)

1867019

In [14]:
def get_flag_counts(catalog):
    unique,count = np.unique(catalog['flags'],return_counts=True)
    return(dict(zip(unique,count)) )

In [15]:
print(get_flag_counts(mcal2))
print(get_flag_counts(mcal3))

{0: 3356849, 1: 47, 2: 159, 4: 885653, 67108864: 93686, 134217728: 28959}
{0: 4290039, 1: 45, 2: 349, 4: 1627, 67108864: 16092, 134217728: 29906}


In [16]:
for number in np.unique(np.array(mcal2['flags'][:])):
    print ("{}".format(np.base_repr(number)))

0
1
10
100
100000000000000000000000000
1000000000000000000000000000


In [17]:
## function to extract fits and hdf data
def load_true():
    """This is a function that takes fits and hdf data and return them to pandas dataframe"""

    # detect = fits.open('/lsst/troxel/balrog/balrog_detection_catalog-v1.2.fits')[-1]
    phot   = fits.open('/lsst/troxel/balrog/balrog_matched_catalog_flatten.fits')[1]
    id_balrog = fits.open('/lsst/troxel/balrog/balrog_mcal_bal_ids_v1.2.fits')

    #load true catalogs in pandas
    # detectp_table=Table.read('/lsst/troxel/balrog/balrog_detection_catalog-v1.2.fits').to_pandas()
    photdf=Table.read('/lsst/troxel/balrog/balrog_matched_catalog_flatten.fits').to_pandas()
    id_balrogdf = Table.read('/lsst/troxel/balrog/balrog_mcal_bal_ids_v1.2.fits').to_pandas()
    
    photdf.drop_duplicates(['bal_id'],inplace=True)
    
    photdf.set_index('bal_id',inplace=True,verify_integrity=True)

    return(photdf)


def load_mcal():
    #all objects riz
#     mcal0= h5py.File('/lsst/troxel/balrog/balrog_mcal_stack-y3v02-0-riz-mcal-v1.2.h5','r')['catalog']['unsheared'] #with neighbor
#     mcal1= h5py.File('/lsst/troxel/balrog/balrog_mcal_stack-y3v02-0-riz-noNB-mcal-v1.2.h5','r')['catalog']['unsheared'] #no neighbor
    ##only matched balrog object riz
    mcal2=h5py.File('/lsst/troxel/balrog/balrog_mcal_stack-y3v02-0-riz-mcal-1.3.h5','r')['catalog']['unsheared'] #with neighbor
    mcal3=h5py.File('/lsst/troxel/balrog/balrog_mcal_stack-y3v02-0-riz-noNB-mcal-1.3.h5','r')['catalog']['unsheared'] #no neighbor
    #only matched balrog object griz
    mcal4=h5py.File('/lsst/troxel/balrog/balrog_mcal_stack-y3v02-0-griz-mcal-1.3.h5','r')['catalog']['unsheared'] #with neighbor
    mcal5=h5py.File('/lsst/troxel/balrog/balrog_mcal_stack-y3v02-0-griz-noNB-mcal-1.3.h5','r')['catalog']['unsheared'] #no neighbor
    
    mcal_list=[mcal2,mcal3,mcal4,mcal5]
    return (mcal_list)

In [18]:
## function to convert 
def get_df_list(mcal_list):
    mcal_df_list =[]
    for catalog in mcal_list:
        local_df=pd.DataFrame()
        for key in catalog.keys():
            local_df[key]=np.array(catalog[key][:]).byteswap().newbyteorder()
        if 'bal_id' in local_df.keys():
            local_df.set_index('bal_id',inplace=True,verify_integrity=True)
        mcal_df_list.append(local_df)
    return(mcal_df_list)

In [19]:
photdf=load_true()

In [20]:
mcal_df_list=get_df_list(load_mcal())

## List of dtypes in mcal

In [21]:
mcal_df_list[2].dtypes

R11                    >f8
R12                    >f8
R21                    >f8
R22                    >f8
T                  float64
T_err              float64
coadd_object_id      int64
covmat_0_1         float64
covmat_1_1         float64
covmat_2_2         float64
dec                float64
e_1                float64
e_2                float64
flags                int32
flux_err_g         float64
flux_err_i         float64
flux_err_r         float64
flux_err_z         float64
flux_g             float64
flux_i             float64
flux_r             float64
flux_z             float64
mask_frac          float64
mcal_psf_T         float64
mcal_psf_e1        float64
mcal_psf_e2        float64
nimage_tot_g         int32
nimage_tot_i         int32
nimage_tot_r         int32
nimage_tot_z         int32
nimage_use_g         int32
nimage_use_i         int32
nimage_use_r         int32
nimage_use_z         int32
psf_T              float64
psf_e1             float64
psf_e2             float64
r

## List of dtypes in phot

In [22]:
photdf.dtypes

true_id                    int32
true_number                int32
true_flags                 int32
true_obj_flags             int32
true_cm_mof_flags          int32
true_ra                  float64
true_dec                 float64
true_cm_T                float64
true_cm_T_err            float64
true_cm_s2n_r            float64
true_cm_flags              int32
true_cm_fracdev          float64
true_cm_TdByTe           float32
true_in_VHS_footprint      int32
true_mag_auto_det        float32
true_det_number            int32
true_tilename             object
meas_id                    int64
meas_number                int32
meas_ra                  float64
meas_dec                 float64
meas_fofid                 int64
meas_flags                 int32
meas_time_last_fit       float64
meas_box_size              int16
meas_obj_flags             int32
meas_mask_frac           float64
meas_psfrec_T            float64
meas_cm_flags              int32
meas_cm_T                float64
          

## sample of mcal data

# Data manipulation

## Constants

In [23]:
SNR_CUT = 10
SIZE_RATIO_CUT = 0.5
T_CUT = 1000

## Filter by SNR T and size_ratio

In [24]:
def filter_by_snr():
    for i,catalog in enumerate(mcal_df_list):
        mcal_df_list[i]=catalog[(catalog['snr']>SNR_CUT) & (catalog['T']<T_CUT) & (catalog['size_ratio']>SIZE_RATIO_CUT)]

In [25]:
filter_by_snr()

## Intersects paris of catalogs by bal_id. We only want objects existing in true catalog, MOF and woMOF  

In [32]:
index_2_3=mcal_df_list[0].index.intersection(mcal_df_list[1].index)
index_4_5=mcal_df_list[2].index.intersection(mcal_df_list[3].index)

##Select objects with true value
index_2_3_true=index_2_3.intersection(photdf.index).sort_values()
index_4_5_true=index_4_5.intersection(photdf.index).sort_values()

In [33]:
mcal_df_list[0]=mcal_df_list[0].reindex(index_2_3_true);
mcal_df_list[1]=mcal_df_list[1].reindex(index_2_3_true);
mcal_df_list[2]=mcal_df_list[2].reindex(index_4_5_true);
mcal_df_list[3]=mcal_df_list[3].reindex(index_4_5_true);

In [34]:
true_df_list=[None]*4
for i in range(2):
    true_df_list[i]=photdf.reindex(index_2_3_true)
    true_df_list[i+2]=photdf.reindex(index_4_5_true)

## Divings catalogs into smaller catalogs only with positive g and only with negative g

In [30]:
mcal_df_list_positive=[None]*4
mcal_df_list_negative=[None]*4

index_2_3_true_positive=index_2_3_true.intersection(photdf[photdf['true_cm_g_1']>0].index.intersection(photdf[photdf['true_cm_g_2']>0].index))
index_2_3_true_negative=index_2_3_true.intersection(photdf[photdf['true_cm_g_1']<0].index.intersection(photdf[photdf['true_cm_g_2']<0].index))

index_4_5_true_positive=index_4_5_true.intersection(photdf[photdf['true_cm_g_1']>0].index.intersection(photdf[photdf['true_cm_g_2']>0].index))
index_4_5_true_negative=index_4_5_true.intersection(photdf[photdf['true_cm_g_1']<0].index.intersection(photdf[photdf['true_cm_g_2']<0].index))

for i in range(2):
    mcal_df_list_positive[i]=mcal_df_list[i].reindex(index_2_3_true_positive)
    mcal_df_list_positive[i+2]=mcal_df_list[i+2].reindex(index_4_5_true_positive)
    mcal_df_list_negative[i]=mcal_df_list[i].reindex(index_2_3_true_negative)
    mcal_df_list_negative[i+2]=mcal_df_list[i+2].reindex(index_4_5_true_negative)

In [42]:
final_df_list = [None]*4
for i,catalog in enumerate(mcal_df_list):
    final_df_list[i]=pd.concat([catalog,photdf[['true_cm_g_1','true_cm_g_2']]],axis=1,join='inner')

In [43]:
final_df_list[0]

,R11,R12,R21,R22,T,T_err,coadd_object_id,covmat_0_1,covmat_1_1,covmat_2_2,...,nimage_use_r,nimage_use_z,psf_T,psf_e1,psf_e2,ra,size_ratio,snr,true_cm_g_1,true_cm_g_2
bal_id,,,,,,,,,,,,,,,,,,,,,
100002002078,1.109e+00,-2.117e-01,-1.953e-01,9.326e-01,5.636e-01,2.799e-02,1240,-3.103e-06,5.926e-04,5.979e-04,...,3,2,3.446e-01,8.422e-04,1.574e-02,7.408e-01,1.636e+00,3.671e+01,-3.415e-02,4.194e-02
100002041238,8.305e-01,2.645e-03,7.385e-04,8.374e-01,1.144e+00,1.661e-03,29227,-1.304e-10,5.236e-07,5.236e-07,...,4,6,4.416e-01,-1.620e-02,3.034e-02,3.970e-01,2.591e+00,2.292e+03,4.112e-02,-2.352e-02
100002042490,1.437e+00,-1.105e+00,-1.148e+00,2.731e+00,7.913e-01,9.006e-02,23493,6.806e-06,1.411e-03,1.419e-03,...,2,4,3.299e-01,-1.984e-03,1.944e-02,6.565e-01,2.398e+00,1.643e+01,-8.739e-02,-1.372e-01
100002042496,8.740e-01,-2.954e-02,-3.613e-02,1.060e+00,5.133e+00,9.691e-03,11311,4.299e-10,4.001e-07,3.985e-07,...,3,3,4.109e-01,-5.885e-03,2.226e-02,6.602e-01,1.249e+01,6.897e+02,3.153e-01,5.591e-02
100002042497,7.362e-01,3.796e-01,2.599e-01,1.001e+00,2.812e-01,2.187e-02,12055,-2.878e-07,1.335e-03,1.334e-03,...,4,4,3.550e-01,-8.721e-03,2.626e-02,1.099e+00,7.920e-01,3.649e+01,1.195e-01,-2.010e-03
100002059578,-1.386e-01,5.240e-01,-1.473e-01,8.055e-04,2.131e-01,3.918e-02,24010,-1.375e-03,6.070e-03,5.213e-03,...,4,3,3.366e-01,-1.675e-02,3.732e-03,9.791e-01,6.330e-01,1.252e+01,6.764e-01,6.887e-02
100003037062,9.384e+00,5.422e+00,-3.261e+00,-2.232e+00,9.006e-01,9.816e-02,33370,-2.185e-04,2.517e-03,2.931e-03,...,4,5,5.236e-01,-6.454e-03,-1.900e-02,8.960e-01,1.720e+00,1.128e+01,3.136e-01,3.629e-01
100004047485,1.255e+00,2.609e-01,7.183e-01,-2.195e+00,1.942e-01,4.904e-02,6025,-6.323e-04,9.135e-03,1.020e-02,...,4,4,3.780e-01,-1.314e-02,2.295e-02,1.032e+00,5.138e-01,1.584e+01,-2.181e-01,5.287e-01
100004047488,8.249e-01,-3.622e-02,-4.505e-02,1.029e+00,5.900e-01,1.278e-02,14964,-7.751e-07,1.038e-04,1.066e-04,...,2,2,3.664e-01,-1.377e-02,1.944e-02,1.443e+00,1.610e+00,1.030e+02,-6.796e-02,1.317e-01
